# BitFields

Because CoHDL implements vector slices as references, it is fairly easy to build abstraction layers on top of the provided primitives.

In [1]:
from cohdl import BitVector

# example class that takes a bitvector as input
# and defines alias names for its elements
class BitAccess:
    def __init__(self, input: BitVector[32]):
        self.first_bit = input[0]
        self.second_bit = input[1]
        self.sub_vec = input[24:12]
        self.sub_unsigned = input[31:20].unsigned

The CoHDL standard library defines the base class `BitField`. It inspects the type annotations of derived types and adds an `__init__` function and assignment operators to the class.

In [2]:
from cohdl import std
from cohdl.std.bitfield import BitField, Field

# define a custom bitfield type with a width of 32 bits
class MyBitfield(BitField[32]):
    # use names a,b and c for bit at offset 0, 1 and 2
    a: Field[0]
    b: Field[1]
    c: BitField.Field[2]    # BitField.Field is an alias for std.bitfield.Field

    # low and high are subvectors
    low: Field[15:0]
    high: Field[31:16]

    # fields can overlap and be Signed or Unsigned
    mid: Field[24:8].Unsigned

In [3]:
from cohdl import Port, BitVector
from cohdl import Entity

class MyEntity(Entity):
    data_in = Port.input(BitVector[32])
    data_out = Port.output(BitVector[32])

    def architecture(self):
        # create bitfield objects from data ports
        # this does not create any new signals
        # the members of inp and out are references
        # to bits/subvectors of self.data_in and self.data_out
        inp = MyBitfield(self.data_in)
        out = MyBitfield(self.data_out)

        @std.sequential
        def logic():
            nonlocal out

            if inp.a & inp.b & inp.c:
                # swap low and high
                out.low <<= inp.high
                out.high <<= inp.low
            elif inp.a:
                # the bitfield decorator defines
                # assignment operators
                out <<= inp
            else:
                # out.mid is unsigned, can use arithmetic operators
                out.mid <<= out.mid + 1


vhdl = std.VhdlCompiler.to_string(MyEntity)
print(vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all;


entity MyEntity is
  port (
    data_in : in std_logic_vector(31 downto 0);
    data_out : out std_logic_vector(31 downto 0)
    );
end MyEntity;


architecture arch_MyEntity of MyEntity is
  function cohdl_bool_to_std_logic(inp: boolean) return std_logic is
  begin
    if inp then
      return('1');
    else
      return('0');
    end if;
  end function cohdl_bool_to_std_logic;
  signal buffer_data_out : std_logic_vector(31 downto 0);
begin
  
  -- CONCURRENT BLOCK (buffer assignment)
  data_out <= buffer_data_out;
  

  logic: process(data_in, buffer_data_out)
    variable temp : std_logic;
    variable temp1 : std_logic;
    variable temp2 : boolean;
    variable temp3 : boolean;
    variable temp4 : unsigned(16 downto 0);
  begin
    temp := (data_in(0)) and (data_in(1));
    temp1 := (temp) and (data_in(2));
    temp2 := temp1 = '1';
    if temp2 then
      buffer_data_out(15 downto 0) <= std_logic_vector(data_